In [ ]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import cv2

import os
import random
# Compare saved image

In [519]:
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3, EfficientNetB0, MobileNetV3Small, MobileNetV2
from tensorflow.keras.applications import resnet50, vgg16, inception_v3, mobilenet_v2, mobilenet_v3

#model_names = ("ResNet50", "VGG", "MobileNetV3Small")
#models = (ResNet50(weights='imagenet'), MobileNetV2(weights='imagenet'), MobileNetV3Small(weights='imagenet'), VGG16(weights='imagenet'))
#applications = (resnet50, mobilenet_v2, mobilenet_v3, vgg16)

all_models = {
    "ResNet50": (ResNet50(weights='imagenet'), resnet50.preprocess_input),
    "VGG16": (VGG16(weights='imagenet'), vgg16.preprocess_input),
    "MobileNetV2": (MobileNetV2(weights='imagenet'), mobilenet_v2.preprocess_input),
    # "MobileNetV3Small": (MobileNetV3Small(weights='imagenet'), mobilenet_v3.preprocess_input),
    "InceptionV3": (InceptionV3(weights='imagenet'), inception_v3.preprocess_input),
    }

/home/paul/anaconda3/envs/nlp/lib/python3.12/site-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


In [22]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
pretrained_model.trainable = False

# ImageNet labels
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [402]:
def preprocess(image, preprocess_input, size=(224,224)):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, size)
  image = preprocess_input(image)
  image = image[None, ...]
  return image

# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
  return decode_predictions(probs, top=1)[0][0]

def display_images(image, description, model):
  _, label, confidence = get_imagenet_label(model.predict(image))
  plt.figure()
  plt.imshow(image[0]*0.5+0.5)
  plt.title('{} \n {} : {:.2f}% Confidence'.format(description,
                                                   label, confidence*100))
  plt.show()


def display_perturbation(pertubation):
  plt.imshow(pertubation * 0.5 + 0.5)
  plt.show()
  

def tensor_range(tensor):
  print(f"Min: {tf.reduce_min(tensor)}, Max: {tf.reduce_max(tensor)}")

In [ ]:
def save_image(tensor, target_folder, filename):

    # Remove batch dimension if present
    if len(tensor.shape) == 4:
        tensor = tensor[0]

    # Encode as JPEG
    encoded = tf.io.encode_jpeg(tensor)

    # Ensure the directory exists
    os.makedirs(target_folder, exist_ok=True)

    # Save to file
    path = os.path.join(target_folder, filename)
    tf.io.write_file(path, encoded)
    print(f"Image saved to: {path}")


def load_and_preprocess_image(path):
    raw = tf.io.read_file(path)
    img = tf.image.decode_image(raw, channels=3)
    return tf.cast(img, tf.float32)

In [461]:
from tensorflow.keras.applications.resnet50 import preprocess_input as res_pre, decode_predictions as res_decode
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_pre, decode_predictions as vgg_decode
from tensorflow.keras.applications.inception_v3 import preprocess_input as inc_pre, decode_predictions as inc_decode
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mn_pre, decode_predictions as mn_decode
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mn2_pre, decode_predictions as mn2_decode

model_names = ("ResNet50", "VGG", "InceptionV3", "MobileNetV3Small", "MobileNetV2")
models = (ResNet50(weights='imagenet'), VGG16(weights='imagenet'), InceptionV3(weights='imagenet'), MobileNetV3Small(weights='imagenet'), MobileNetV2(weights='imagenet'))
pre_functions = (res_pre, vgg_pre, inc_pre, mn_pre, mn2_pre)
decode_functions = (res_decode, vgg_decode, inc_decode, mn_decode, mn2_decode)

/home/paul/anaconda3/envs/nlp/lib/python3.12/site-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


In [460]:
def plot_histogram(data, total, label, title, target_label = None):
    if target_label != None:
        plt.bar([label, target_label, "other"], [data[label], data[target_label], total - data[label] - data[target_label]])
        print(f"Targeted evasion rate {((data[target_label]) / total) * 100: .2f}")
    else:
        plt.bar([label, "other"], [data[label], total - data[label]])
        print(f"Evasion rate {((total - data[label]) / total) * 100: .2f}")
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.title('Histogram: ' + title)
    plt.show()


def test_model(img_tensors, model, preprocess, decode, flag):
    l = {}
    target_size = (299, 299) if flag == 1 else (224, 224)

    processed_images = []
    for image in img_tensors:
        image = tf.image.resize(image, target_size)
        image = tf.cast(image, tf.float32)
        processed_images.append(image)

    batch = tf.stack(processed_images)
    batch = preprocess(batch)
    predictions = model.predict(batch)
    decoded = decode(predictions, top=1)
    
    for d in decoded:
        _, label, conf = d[0]
        # print(f"Predicted: {label} with {conf*100:.2f}% confidence")
        if label not in l.keys():
            l[label] = 1
        else:
            l[label] += 1

    return l

## Targeted Attacks

In [604]:
# For clipping the input vector
def clip_delta(t, eps):
    return tf.clip_by_value(t, -eps, eps)


# Generates targeted adversarial images
def generate_adv_pertubation(model_name, model, preprocessor, optimizer, loss, img, delta, label, target_label, eps, max_iterations=500):
    # delta = tf.Variable(tf.zeros_like(img), trainable=True)
    
    for step in range(max_iterations):
        with tf.GradientTape() as tape:
            tape.watch(delta)
            
            if model_name == "InceptionV3":
                adv = preprocess(img + delta, preprocessor, (299, 299))
            else:
                adv = preprocess(img + delta, preprocessor)

            preds = model(adv,training=False)
            # print(preds)
            originalLoss = -loss(tf.convert_to_tensor([label]), preds)
            targetLoss = loss(tf.convert_to_tensor([target_label]), preds)
            totalLoss = originalLoss + targetLoss

            # if step % 50 == 0:
            #     print("step: {}, loss: {}...".format(step, totalLoss.numpy()), "Range:", tf.reduce_min(adv).numpy(), "to", tf.reduce_max(adv).numpy())

            gradients = tape.gradient(totalLoss, delta)
            optimizer.apply_gradients([(gradients, delta)])
            delta.assign_add(clip_delta(delta, eps))
    # tensor_range(delta)
    
    return delta

In [605]:
def generate_adversarial_images(images, models, source_folder, target_folder, label, target_label, eps, iterations):
    i, mx = 0, len(images)
    adversarial_imgs = []
    for img_name in images:
        print(f"{i}/{mx}: {img_name}")
        
        image_raw = tf.io.read_file(source_folder + img_name)
        image = tf.image.decode_image(image_raw)
        
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        adam = tf.keras.optimizers.Adam(1e-2)
        
        baseImage = tf.constant(image / 1, dtype=tf.float32)
        delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)

        for m in models.keys():
            
            model = models[m][0]
            preprocessor = models[m][1]

            target_pertubation = generate_adv_pertubation(m, model, preprocessor, adam, loss, baseImage, delta, label, target_label, eps, iterations)
            adv = (baseImage + target_pertubation)

            # baseImage = tf.clip_by_value(adv, val1, val2)
            baseImage = adv

        if m == "InceptionV3":
            final = preprocess(adv, preprocessor, (299, 299))
        else:
            final = preprocess(adv, preprocessor)
        _, cl, conf = get_imagenet_label(model.predict(final))
        # display_images(adv, descriptions[i])
        print(f"Classified as {cl} with {conf}% confidence.")
        
        # Convert from [-1, 1] to [0, 255]
        tensor = (final + 1.0) / 2.0
        tensor = tf.clip_by_value(tensor, 0.0, 1.0)
        tensor = tf.image.convert_image_dtype(tensor, dtype=tf.uint8)
        save_image(tensor, target_folder, img_name)
        adversarial_imgs.append(tensor)
        i += 1
    return adversarial_imgs

In [ ]:
full_path = "./banana_images/data/train/banana/"
target_path = "./adversarial_images/targeted"
image_files = [f for f in os.listdir(full_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

sub_images = image_files[:100]

model_name = "MobileNet2"
model = {model_name: all_models[model_name]}

eps = 0.03
label = tf.Variable(954)
target_label = tf.Variable(953)
iterations = 300

advs_targeted = generate_adversarial_images(sub_images, model, full_path, target_path, label, target_label, eps, iterations)

0/100: BXGWH5LDSBVH.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Classified as pineapple with 0.7180190682411194% confidence.
Image saved to: ./adversarial_images/targeted/BXGWH5LDSBVH.jpg
1/100: 0M5JU924BDT7.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Classified as pineapple with 0.9999953508377075% confidence.
Image saved to: ./adversarial_images/targeted/0M5JU924BDT7.jpg
2/100: 67IKPXNTBPW4.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Classified as pineapple with 1.0% confidence.
Image saved to: ./adversarial_images/targeted/67IKPXNTBPW4.jpg
3/100: ZP0SPW9WVAXD.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Classified as pineapple with 0.9996436834335327% confidence.
Image saved to: ./adversarial_images/targeted/ZP0SPW9WVAXD.jpg
4/100: MRBF5A4A1CJA.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Classified as pineapple with 0.9174841642379761% confidence.
Image saved to: ./adversarial_images/targeted/MRBF5A4A1CJA.jpg
5/100: P7UIOT2KVDET.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Classified as pineapple w

In [607]:
full_path = "./banana_images/data/train/banana/"
target_path = "./adversarial_images/targeted2"
image_files = [f for f in os.listdir(full_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

sub_images = image_files[:1]

model_name = "InceptionV3"
model = {model_name: all_models[model_name]}

eps = 0.03
label = tf.Variable(954)
target_label = tf.Variable(859)
iterations = 10

advs_targeted = generate_adversarial_images(sub_images, model, full_path, target_path, label, target_label, eps, iterations)

0/1: BXGWH5LDSBVH.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Classified as banana with 0.7340413928031921% confidence.
Image saved to: ./adversarial_images/targeted2/BXGWH5LDSBVH.jpg


## Untargeted attacks

In [7]:
def create_adversarial_pattern(input_image, input_label, loss_object, model):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

In [ ]:
def generate_untargeted_adversarial_images(images, source_folder, target_folder, label, eps):
    i, mx = 0, len(images)
    label = tf.one_hot(label, 1000)
    label = tf.reshape(label, (1, 1000))
    for img_name in images:
        print(f"{i}/{mx}: {img_name}")

        image_path = tf.keras.utils.get_file(img_name, 'file://' + source_folder + img_name)
        image_raw = tf.io.read_file(image_path)
        image = tf.image.decode_image(image_raw)

        image_pr = preprocess(image)
        
        loss_object = tf.keras.losses.CategoricalCrossentropy()

        pertubation = create_adversarial_pattern(image_pr, label, loss_object, pretrained_model)

        adv = image_pr + eps * pertubation
        adv = tf.clip_by_value(adv, -1, 1)

        _, cl, conf = get_imagenet_label(pretrained_model.predict(adv))
        # display_images(adv, descriptions[i])
        print(f"Classified as {cl} with {conf}% confidence.")
        
        save_image(adv, target_folder, img_name)
        i += 1

In [587]:
def deprocess_vgg(image):
    # Assume shape (H, W, 3) or (1, H, W, 3)
    if image.shape.rank == 4:
        image = image[0]
    image = image[..., ::-1]  # BGR → RGB
    image = image + [103.939, 116.779, 123.68]  # Unsubtract means
    image = tf.clip_by_value(image / 255.0, 0.0, 1.0)
    return image

def deprocess_resnet(image):
    if len(image.shape) == 4:
        image = image[0]
    
    image = image[..., ::-1]  # BGR → RGB
    image = image + [103.939, 116.779, 123.68]  # Unsubtract means
    image = tf.clip_by_value(image, 0.0, 255.0)
    return image / 255.0  # normalize for display


def generate_untargeted_adversarial_images_multiple_models(images, source_folder, target_folder, label, eps, models):

    i, mx = 0, len(images)
    label = tf.one_hot(label, 1000)
    label = tf.reshape(label, (1, 1000))
    adversarial_tensors = []

    for img_name in images:
        print(f"{i + 1}/{mx}: {img_name}")

        # image_path = tf.keras.utils.get_file(img_name, 'file://' + source_folder + img_name)
        image_raw = tf.io.read_file(source_folder + img_name)
        image = tf.image.decode_image(image_raw)
        for j, m in enumerate(models.keys()):
            model = models[m][0]
            preprocessor = models[m][1]
            
            if m == "InceptionV3":
                image_pr = preprocess(image, preprocessor, (299, 299))
            elif m == "MobileNetV3" or m == "MobileNetV3Small":
                image_pr = preprocess(image, lambda x: x)
            else:
                image_pr = preprocess(image, preprocessor)

            # plt.imshow(image_pr[0])
            # plt.title("After preprocessing")
            # plt.show()

            loss_object = tf.keras.losses.CategoricalCrossentropy()
            pertubation = create_adversarial_pattern(image_pr, label, loss_object, model)
            # plt.imshow(pertubation[0] * 0.5 + 0.5)
            # plt.title(f"Perturbation with {m}")
            # plt.show()

            if m == "VGG16" or m == "ResNet50":
                adv = image_pr + 100 * eps * pertubation
                adv = tf.clip_by_value(adv, -128, 128)
                final = (deprocess_resnet(adv) - 0.5) * 2

                # tensor_range(depr)
                # plt.imshow(depr)
                # plt.show()
            elif m == "MobileNetV2" or m == "MobileNetV3Small" or m == "InceptionV3":
                adv = image_pr + eps * pertubation
                final = tf.clip_by_value(adv, -1, 1)
            elif m == "MobileNetV3" or m == "MobileNetV3Small":
                adv = image_pr + 255 * eps * pertubation
                final = tf.clip_by_value(adv, 0, 255)

            # Convert from [-1, 1] to [0, 255]
            tensor = (final + 1.0) / 2.0
            tensor = tf.clip_by_value(tensor, 0.0, 1.0)
            tensor = tf.image.convert_image_dtype(tensor, dtype=tf.uint8)

            if len(tensor.shape) == 4:
                tensor = tensor[0]
            # plt.imshow(tensor)
            # plt.title(f"After perturbation with model {m}")
            # plt.show()

            image = tensor
        save_image(tensor, target_folder, img_name)
        adversarial_tensors.append(tensor)
        # _, cl, conf = get_imagenet_label(all_models["MobileNetV2"][0].predict(preprocess(tensor, all_models["MobileNetV2"][1])))
        # print(f"Classified as {cl} with {conf}% confidence.")

        i += 1
        
    return adversarial_tensors

In [588]:
full_path = "./banana_images/data/train/banana/"
target_path = "./adversarial_images/untargeted/"
image_files = [f for f in os.listdir(full_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:100]

sub_images = image_files

model_name = "ResNet50"
model = {model_name: all_models[model_name]}
eps = 0.01
label = tf.Variable(954)


advs = generate_untargeted_adversarial_images_multiple_models(sub_images, full_path, target_path, label, eps, model)
# tensor_range(advs[0])
# advs[0]

1/100: BXGWH5LDSBVH.jpg
Image saved to: ./adversarial_images/untargeted/BXGWH5LDSBVH.jpg
2/100: 0M5JU924BDT7.jpg
Image saved to: ./adversarial_images/untargeted/0M5JU924BDT7.jpg
3/100: 67IKPXNTBPW4.jpg
Image saved to: ./adversarial_images/untargeted/67IKPXNTBPW4.jpg
4/100: ZP0SPW9WVAXD.jpg
Image saved to: ./adversarial_images/untargeted/ZP0SPW9WVAXD.jpg
5/100: MRBF5A4A1CJA.jpg
Image saved to: ./adversarial_images/untargeted/MRBF5A4A1CJA.jpg
6/100: P7UIOT2KVDET.jpg
Image saved to: ./adversarial_images/untargeted/P7UIOT2KVDET.jpg
7/100: NWQ45BLV4HUP.jpg
Image saved to: ./adversarial_images/untargeted/NWQ45BLV4HUP.jpg
8/100: 9G3QHFUWLQ22.jpg
Image saved to: ./adversarial_images/untargeted/9G3QHFUWLQ22.jpg
9/100: 74EEYDBZQQSJ.jpg
Image saved to: ./adversarial_images/untargeted/74EEYDBZQQSJ.jpg
10/100: WVGPEY52XK9O.jpg
Image saved to: ./adversarial_images/untargeted/WVGPEY52XK9O.jpg
11/100: O2TH8EXK41KR.jpg
Image saved to: ./adversarial_images/untargeted/O2TH8EXK41KR.jpg
12/100: ID4B9VD0XBK

In [517]:
full_path = "./banana_images/data/train/banana/"
target_path = "./adversarial_images/untargeted2/"
image_files = [f for f in os.listdir(full_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:100]

sub_images = image_files

# model_name = "MobileNetV2"
# model = {model_name: all_models[model_name]}
eps = 0.005
label = tf.Variable(954)

advs = generate_untargeted_adversarial_images_multiple_models(sub_images, full_path, target_path, label, eps, all_models)

1/100: BXGWH5LDSBVH.jpg
Image saved to: ./adversarial_images/untargeted2/BXGWH5LDSBVH.jpg
2/100: 0M5JU924BDT7.jpg
Image saved to: ./adversarial_images/untargeted2/0M5JU924BDT7.jpg
3/100: 67IKPXNTBPW4.jpg
Image saved to: ./adversarial_images/untargeted2/67IKPXNTBPW4.jpg
4/100: ZP0SPW9WVAXD.jpg
Image saved to: ./adversarial_images/untargeted2/ZP0SPW9WVAXD.jpg
5/100: MRBF5A4A1CJA.jpg
Image saved to: ./adversarial_images/untargeted2/MRBF5A4A1CJA.jpg
6/100: P7UIOT2KVDET.jpg
Image saved to: ./adversarial_images/untargeted2/P7UIOT2KVDET.jpg
7/100: NWQ45BLV4HUP.jpg
Image saved to: ./adversarial_images/untargeted2/NWQ45BLV4HUP.jpg
8/100: 9G3QHFUWLQ22.jpg
Image saved to: ./adversarial_images/untargeted2/9G3QHFUWLQ22.jpg
9/100: 74EEYDBZQQSJ.jpg
Image saved to: ./adversarial_images/untargeted2/74EEYDBZQQSJ.jpg
10/100: WVGPEY52XK9O.jpg
Image saved to: ./adversarial_images/untargeted2/WVGPEY52XK9O.jpg
11/100: O2TH8EXK41KR.jpg
Image saved to: ./adversarial_images/untargeted2/O2TH8EXK41KR.jpg
12/100: 